In [8]:
from torch.optim import Adam, SGD
import torch.nn as nn
import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from sklearn import preprocessing
from inverse_modelling_tfo.models import train_model, train_model_wtih_reporting
from inverse_modelling_tfo.data import generate_data_loaders, equidistance_detector_normalization, constant_detector_count_normalization
from inverse_modelling_tfo.data.intensity_interpolation import get_interpolate_fit_params_custom, interpolate_exp
from inverse_modelling_tfo.data.interpolation_function_zoo import *
from inverse_modelling_tfo.models.custom_models import SplitChannelCNN, PerceptronReLU
from inverse_modelling_tfo.features.build_features import create_ratio, create_spatial_intensity
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
import torchinfo
# Set my GPU
os.environ["CUDA_VISIBLE_DEVICES"]="2"

# data = pd.read_pickle(r'/home/rraiyan/personal_projects/tfo_inverse_modelling/data/intensity/s_based_intensity.pkl')
# data = pd.read_pickle(r'/home/rraiyan/personal_projects/tfo_inverse_modelling/data/intensity/s_based_intensity_low_conc.pkl')
data = pd.read_pickle(r'/home/rraiyan/personal_projects/tfo_inverse_modelling/data/intensity/s_based_intensity_low_conc2.pkl')
equidistance_detector_normalization(data)

# Drop Uterus Thickness for now
data = data.drop(columns='Uterus Thickness')

# Interpolate intensity to remove noise
data = interpolate_exp(data, weights=[1, -1])
data['Intensity'] = data['Interpolated Intensity']
data = data.drop(columns='Interpolated Intensity')

# Manual log(intensity) normalization
data['Intensity'] = np.log10(data['Intensity'])        # Far values wayy to small to affect anything. Take log
data = create_spatial_intensity(data)
data.head()

,Maternal Wall Thickness,Maternal Hb Concentration,Maternal Saturation,Fetal Hb Concentration,Fetal Saturation,10_1.0,14_1.0,19_1.0,23_1.0,28_1.0,...,55_2.0,59_2.0,64_2.0,68_2.0,73_2.0,77_2.0,82_2.0,86_2.0,91_2.0,95_2.0
0,2.0,12.0,0.9,0.11,0.100,-4.784362,-5.887022,-6.560915,-7.015157,-7.712442,...,-19.639869,-22.003357,-25.265887,-28.112349,-31.952534,-35.240710,-39.608559,-43.300212,-48.150422,-52.211376
1,2.0,12.0,0.9,0.11,0.225,-4.783804,-5.882596,-6.550246,-6.998650,-7.687817,...,-19.677858,-22.046210,-25.314961,-28.166499,-32.013139,-35.306559,-39.681050,-43.378082,-48.235089,-52.301534
2,2.0,12.0,0.9,0.11,0.350,-4.783210,-5.877882,-6.538860,-6.980969,-7.661314,...,-19.715011,-22.088109,-25.362928,-28.219417,-32.072350,-35.370881,-39.751845,-43.454117,-48.317744,-52.389537
3,2.0,12.0,0.9,0.11,0.475,-4.782576,-5.872834,-6.526638,-6.961915,-7.632599,...,-19.751362,-22.129094,-25.409836,-28.271155,-32.130227,-35.433740,-39.821015,-43.528395,-48.398473,-52.475475
4,2.0,12.0,0.9,0.11,0.600,-4.781896,-5.867394,-6.513429,-6.941227,-7.601231,...,-19.786945,-22.169203,-25.455727,-28.321760,-32.186822,-35.495197,-39.888626,-43.600986,-48.477353,-52.559433


In [16]:
train_split = 0.8
table = data

row_ids = np.arange(0, len(table), 1)
np.random.shuffle(row_ids)
train_ids = row_ids[:int(len(row_ids) * train_split)]
validation_ids = row_ids[int(len(row_ids) * train_split):]
a = table.iloc[train_ids, :].copy().reset_index(drop=True)
b = table.iloc[validation_ids, :].copy().reset_index(drop=True)


In [2]:
test = data.groupby(['Maternal Wall Thickness', "Fetal Saturation", "Maternal Saturation", "Maternal Hb Concentration"])
test.head()

,Maternal Wall Thickness,Maternal Hb Concentration,Maternal Saturation,Fetal Hb Concentration,Fetal Saturation,10_1.0,14_1.0,19_1.0,23_1.0,28_1.0,...,55_2.0,59_2.0,64_2.0,68_2.0,73_2.0,77_2.0,82_2.0,86_2.0,91_2.0,95_2.0
0,2.0,12.0,0.9,0.11,0.100,-4.784362,-5.887022,-6.560915,-7.015157,-7.712442,...,-19.639869,-22.003357,-25.265887,-28.112349,-31.952534,-35.240710,-39.608559,-43.300212,-48.150422,-52.211376
1,2.0,12.0,0.9,0.11,0.225,-4.783804,-5.882596,-6.550246,-6.998650,-7.687817,...,-19.677858,-22.046210,-25.314961,-28.166499,-32.013139,-35.306559,-39.681050,-43.378082,-48.235089,-52.301534
2,2.0,12.0,0.9,0.11,0.350,-4.783210,-5.877882,-6.538860,-6.980969,-7.661314,...,-19.715011,-22.088109,-25.362928,-28.219417,-32.072350,-35.370881,-39.751845,-43.454117,-48.317744,-52.389537
3,2.0,12.0,0.9,0.11,0.475,-4.782576,-5.872834,-6.526638,-6.961915,-7.632599,...,-19.751362,-22.129094,-25.409836,-28.271155,-32.130227,-35.433740,-39.821015,-43.528395,-48.398473,-52.475475
4,2.0,12.0,0.9,0.11,0.600,-4.781896,-5.867394,-6.513429,-6.941227,-7.601231,...,-19.786945,-22.169203,-25.455727,-28.321760,-32.186822,-35.495197,-39.888626,-43.600986,-48.477353,-52.559433
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1870,6.0,16.0,1.0,0.17,0.100,-5.031972,-7.252754,-9.452121,-10.377177,-10.418920,...,-20.098004,-18.794543,-16.671396,-14.601150,-11.577282,-8.828676,-5.004744,-1.651133,2.889179,6.786600
1871,6.0,16.0,1.0,0.17,0.225,-5.031972,-7.252707,-9.448460,-10.362078,-10.377225,...,-20.317803,-19.060092,-16.998858,-14.981689,-12.028315,-9.339231,-5.593378,-2.305013,2.150455,5.977502
1872,6.0,16.0,1.0,0.17,0.350,-5.031971,-7.252656,-9.444456,-10.345725,-10.332242,...,-20.532280,-19.319211,-17.318391,-15.353015,-12.468429,-9.837426,-6.167762,-2.943063,1.429614,5.187991
1873,6.0,16.0,1.0,0.17,0.475,-5.031971,-7.252600,-9.440042,-10.327881,-10.283358,...,-20.741667,-19.572180,-17.630341,-15.715528,-12.898097,-10.323798,-6.728514,-3.565971,0.725881,4.417218


In [3]:
for index, df in test:
    pass

In [4]:
df

,Maternal Wall Thickness,Maternal Hb Concentration,Maternal Saturation,Fetal Hb Concentration,Fetal Saturation,10_1.0,14_1.0,19_1.0,23_1.0,28_1.0,...,55_2.0,59_2.0,64_2.0,68_2.0,73_2.0,77_2.0,82_2.0,86_2.0,91_2.0,95_2.0
1854,6.0,16.0,1.0,0.110,0.6,-5.031969,-7.252374,-9.421650,-10.255239,-10.086254,...,-18.272427,-16.588993,-13.951621,-11.440540,-7.831177,-4.588203,-0.115784,3.779733,9.024723,13.506637
1859,6.0,16.0,1.0,0.125,0.6,-5.031969,-7.252420,-9.425425,-10.269953,-10.125956,...,-19.019194,-17.491191,-15.064166,-12.733410,-9.363547,-6.322797,-2.115646,1.558202,6.514940,10.757764
1864,6.0,16.0,1.0,0.140,0.6,-5.031969,-7.252462,-9.428903,-10.283591,-10.162848,...,-19.708835,-18.324373,-16.091604,-13.927382,-10.778699,-7.924705,-3.962530,-0.493396,4.197137,8.219160
1869,6.0,16.0,1.0,0.155,0.6,-5.031970,-7.252502,-9.432127,-10.296309,-10.197339,...,-20.349105,-19.097909,-17.045492,-15.035882,-12.092548,-9.411941,-5.677207,-2.398134,2.045251,5.862276
1874,6.0,16.0,1.0,0.170,0.6,-5.031970,-7.252539,-9.435131,-10.308231,-10.229752,...,-20.946180,-19.819261,-17.935029,-16.069602,-13.317763,-10.798847,-7.276212,-4.174378,0.038530,3.664387


In [5]:
from inverse_modelling_tfo.data.data_loader import DifferentialCombinationDataset
dl = DifferentialCombinationDataset(data, ['Maternal Wall Thickness', "Fetal Saturation", "Maternal Saturation"], x_columns=['10_1.0', '14_1.0'], differential_column='Fetal Hb Concentration', total_length=2000)

In [7]:
for i in range(20):
    a, b = dl.__getitem__(i)
    print(b.shape)

torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])


In [ ]:
dl.randomized_row_indices

array([[18, 14],
       [18,  1],
       [22, 21],
       ...,
       [ 3, 17],
       [ 3,  5],
       [22, 10]])